## Libs

In [1]:
import matplotlib as matpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

# Using the Tensorflow backend (default).
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow import set_random_seed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping

# advanced plotting
import seaborn as sns
plt.style.use('seaborn-darkgrid')
%matplotlib inline

Using TensorFlow backend.


## Data

In [14]:
%store -r data_SHY

In [15]:
mm = MinMaxScaler(feature_range = (0,1))
dataset = mm.fit_transform(data_SHY)

In [16]:
split = 0.6

train_size = int(len(dataset) * split)
#validation
test_size = len(dataset) - train_size

train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

print("training, test set: " + str((len(train), len(test))))

training, test set: (313, 209)


In [17]:
def input_dataset(dataset, window):
    data_X, data_y = [], []
    for i in range(len(dataset) - window - 1):
        a = dataset[i:(i + window), 0]
        data_X.append(a)
        data_y.append(dataset[i + window, 0])
    return(np.array(data_X), np.array(data_y))


In [18]:
# New testing and training sets for rolling forecast.
window = 1
train_X, train_Y = input_dataset(train, window)
test_X, test_Y = input_dataset(test, window)
print("Original train shape:")
print(train_X.shape)

# Reshape input data to match Keras format.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New train shape:")
print(train_X.shape)

Original train shape:
(311, 1)
New train shape:
(311, 1, 1)


## 1 Epoch

In [7]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = 'mean_squared_error', 
                  optimizer = 'adam')
    earlyStop=EarlyStopping(monitor='val_loss',verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

W0919 02:28:12.462113 4604585408 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 02:28:12.463948 4604585408 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 02:28:12.466881 4604585408 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 02:28:12.678099 4604585408 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0919 02:28:12.930221 4604585408 deprecation.py:323] From //anaconda3/li

Epoch 1/1
311/311 [==============================] - 5s 15ms/step - loss: 4.0952e-04


In [8]:
def prediction_score(model, X, Y):
    # Make predictions using input data
    pred = mm.inverse_transform(model.predict(X))
    # Show Y on original scale
    original_data = mm.inverse_transform([Y])
    # RMSE.
    score = math.sqrt(mean_squared_error(original_data[0], pred[:, 0]))
    return(score, pred)

In [9]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 0.57 RMSE


## 1000 Epochs

In [10]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/1000
311/311 [==============================] - 5s 17ms/step - loss: 4.4452e-04
Epoch 2/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0028
Epoch 3/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 4/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.9968e-04
Epoch 5/1000
311/311 [==============================] - 3s 8ms/step - loss: 4.8485e-04
Epoch 6/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.3778e-04
Epoch 7/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.7300e-04
Epoch 8/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.4596e-04
Epoch 9/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.3554e-04
Epoch 10/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.3248e-04
Epoch 11/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.3277e-04
Epoch 12/1000
311/311 [=========================

311/311 [==============================] - 2s 8ms/step - loss: 2.0799e-04
Epoch 95/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0784e-04
Epoch 96/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0771e-04
Epoch 97/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0751e-04
Epoch 98/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0736e-04
Epoch 99/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0716e-04
Epoch 100/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0700e-04
Epoch 101/1000
311/311 [==============================] - 3s 8ms/step - loss: 2.0687e-04
Epoch 102/1000
311/311 [==============================] - 3s 8ms/step - loss: 2.0671e-04
Epoch 103/1000
311/311 [==============================] - 3s 8ms/step - loss: 2.0655e-04
Epoch 104/1000
311/311 [==============================] - 3s 8ms/step - loss: 2.0640e-04
Epoch 105/1000
311/311 [=================

311/311 [==============================] - 3s 9ms/step - loss: 2.0225e-04
Epoch 187/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.0224e-04
Epoch 188/1000
311/311 [==============================] - 3s 8ms/step - loss: 2.0224e-04
Epoch 189/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0223e-04
Epoch 190/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0223e-04
Epoch 191/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0222e-04
Epoch 192/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0221e-04
Epoch 193/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0221e-04
Epoch 194/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0220e-04
Epoch 195/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0220e-04
Epoch 196/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0220e-04
Epoch 197/1000
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 2.0156e-04
Epoch 279/1000
311/311 [==============================] - 3s 8ms/step - loss: 2.0155e-04
Epoch 280/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0153e-04
Epoch 281/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0152e-04
Epoch 282/1000
311/311 [==============================] - 2s 8ms/step - loss: 2.0150e-04
Epoch 283/1000
311/311 [==============================] - 3s 8ms/step - loss: 2.0149e-04
Epoch 284/1000
311/311 [==============================] - 3s 8ms/step - loss: 2.0147e-04
Epoch 285/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.0146e-04
Epoch 286/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.0144e-04
Epoch 287/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.0142e-04
Epoch 288/1000
311/311 [==============================] - 3s 9ms/step - loss: 2.0141e-04
Epoch 289/1000
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 1.9982e-04
Epoch 371/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9980e-04
Epoch 372/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9977e-04
Epoch 373/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9975e-04
Epoch 374/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9973e-04
Epoch 375/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9972e-04
Epoch 376/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9969e-04
Epoch 377/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9967e-04
Epoch 378/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9965e-04
Epoch 379/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9963e-04
Epoch 380/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9961e-04
Epoch 381/1000
311/311 [============

...

311/311 [==============================] - 2s 8ms/step - loss: 1.9656e-04
Epoch 555/1000
311/311 [==============================] - 2s 8ms/step - loss: 1.9655e-04
Epoch 556/1000
311/311 [==============================] - 2s 8ms/step - loss: 1.9654e-04
Epoch 557/1000
311/311 [==============================] - 2s 8ms/step - loss: 1.9653e-04
Epoch 558/1000
311/311 [==============================] - 2s 8ms/step - loss: 1.9651e-04
Epoch 559/1000
311/311 [==============================] - 2s 8ms/step - loss: 1.9650e-04
Epoch 560/1000
311/311 [==============================] - 2s 8ms/step - loss: 1.9649e-04
Epoch 561/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9648e-04
Epoch 562/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9647e-04
Epoch 563/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9646e-04
Epoch 564/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9644e-04
Epoch 565/1000
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 1.9556e-04
Epoch 646/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9556e-04
Epoch 647/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9555e-04
Epoch 648/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9554e-04
Epoch 649/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9553e-04
Epoch 650/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9551e-04
Epoch 651/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9551e-04
Epoch 652/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9550e-04
Epoch 653/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9549e-04
Epoch 654/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9545e-04
Epoch 655/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9546e-04
Epoch 656/1000
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 1.9466e-04
Epoch 738/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9464e-04
Epoch 739/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9463e-04
Epoch 740/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9463e-04
Epoch 741/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9462e-04
Epoch 742/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9461e-04
Epoch 743/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9460e-04
Epoch 744/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9459e-04
Epoch 745/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9458e-04
Epoch 746/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9457e-04
Epoch 747/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9457e-04
Epoch 748/1000
311/311 [============

311/311 [==============================] - 2s 6ms/step - loss: 1.9385e-04
Epoch 830/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9385e-04
Epoch 831/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9384e-04
Epoch 832/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9383e-04
Epoch 833/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9382e-04
Epoch 834/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9381e-04
Epoch 835/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9380e-04
Epoch 836/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9380e-04
Epoch 837/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9379e-04
Epoch 838/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9378e-04
Epoch 839/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.9377e-04
Epoch 840/1000
311/311 [============

311/311 [==============================] - 3s 9ms/step - loss: 1.9314e-04
Epoch 922/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9314e-04
Epoch 923/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9313e-04
Epoch 924/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9312e-04
Epoch 925/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9311e-04
Epoch 926/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9311e-04
Epoch 927/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9310e-04
Epoch 928/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9309e-04
Epoch 929/1000
311/311 [==============================] - 3s 8ms/step - loss: 1.9309e-04
Epoch 930/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9307e-04
Epoch 931/1000
311/311 [==============================] - 3s 9ms/step - loss: 1.9307e-04
Epoch 932/1000
311/311 [============

In [11]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)

print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 0.24 RMSE


## 2000 Epochs

In [13]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 0.31 RMSE


## 2500 Epochs

In [20]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2500, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/2500
311/311 [==============================] - 7s 22ms/step - loss: 4.5241e-04
Epoch 2/2500
311/311 [==============================] - 2s 5ms/step - loss: 0.0029
Epoch 3/2500
311/311 [==============================] - 2s 5ms/step - loss: 0.0015
Epoch 4/2500
311/311 [==============================] - 2s 5ms/step - loss: 8.1852e-04
Epoch 5/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.9183e-04
Epoch 6/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.4003e-04
Epoch 7/2500
311/311 [==============================] - 2s 5ms/step - loss: 2.7356e-04
Epoch 8/2500
311/311 [==============================] - 2s 6ms/step - loss: 2.4604e-04
Epoch 9/2500
311/311 [==============================] - 2s 6ms/step - loss: 2.3552e-04
Epoch 10/2500
311/311 [==============================] - 2s 7ms/step - loss: 2.3250e-04
Epoch 11/2500
311/311 [==============================] - 2s 6ms/step - loss: 2.3289e-04
Epoch 12/2500
311/311 [=========================

311/311 [==============================] - 3s 8ms/step - loss: 2.0751e-04
Epoch 95/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0733e-04
Epoch 96/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0714e-04
Epoch 97/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0705e-04
Epoch 98/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0687e-04
Epoch 99/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0673e-04
Epoch 100/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0655e-04
Epoch 101/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0637e-04
Epoch 102/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0625e-04
Epoch 103/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0610e-04
Epoch 104/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0603e-04
Epoch 105/2500
311/311 [=================

311/311 [==============================] - 3s 8ms/step - loss: 2.0217e-04
Epoch 187/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0217e-04
Epoch 188/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0216e-04
Epoch 189/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0216e-04
Epoch 190/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0216e-04
Epoch 191/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0216e-04
Epoch 192/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0217e-04
Epoch 193/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0216e-04
Epoch 194/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0216e-04
Epoch 195/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0216e-04
Epoch 196/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0216e-04
Epoch 197/2500
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 2.0166e-04
Epoch 279/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0165e-04
Epoch 280/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0163e-04
Epoch 281/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0162e-04
Epoch 282/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0161e-04
Epoch 283/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0159e-04
Epoch 284/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0158e-04
Epoch 285/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0156e-04
Epoch 286/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0155e-04
Epoch 287/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0153e-04
Epoch 288/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0152e-04
Epoch 289/2500
311/311 [============

311/311 [==============================] - 2s 8ms/step - loss: 2.0017e-04
Epoch 371/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0015e-04
Epoch 372/2500
311/311 [==============================] - 2s 8ms/step - loss: 2.0013e-04
Epoch 373/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0012e-04
Epoch 374/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0010e-04
Epoch 375/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0008e-04
Epoch 376/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0006e-04
Epoch 377/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0005e-04
Epoch 378/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0002e-04
Epoch 379/2500
311/311 [==============================] - 3s 8ms/step - loss: 2.0001e-04
Epoch 380/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.9999e-04
Epoch 381/2500
311/311 [============

...

311/311 [==============================] - 3s 8ms/step - loss: 1.8742e-04
Epoch 2101/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8730e-04
Epoch 2102/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8720e-04
Epoch 2103/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8707e-04
Epoch 2104/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8782e-04
Epoch 2105/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8753e-04
Epoch 2106/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8741e-04
Epoch 2107/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8729e-04
Epoch 2108/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8719e-04
Epoch 2109/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8708e-04
Epoch 2110/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8781e-04
Epoch 2111/2500
311/311 [=

311/311 [==============================] - 3s 9ms/step - loss: 1.8772e-04
Epoch 2192/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8759e-04
Epoch 2193/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8746e-04
Epoch 2194/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8735e-04
Epoch 2195/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8722e-04
Epoch 2196/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8799e-04
Epoch 2197/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8770e-04
Epoch 2198/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8758e-04
Epoch 2199/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8745e-04
Epoch 2200/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8723e-04
Epoch 2201/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8808e-04
Epoch 2202/2500
311/311 [=

311/311 [==============================] - 3s 8ms/step - loss: 1.8764e-04
Epoch 2283/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8752e-04
Epoch 2284/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8737e-04
Epoch 2285/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8816e-04
Epoch 2286/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8787e-04
Epoch 2287/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8773e-04
Epoch 2288/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8760e-04
Epoch 2289/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8739e-04
Epoch 2290/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8823e-04
Epoch 2291/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8793e-04
Epoch 2292/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8779e-04
Epoch 2293/2500
311/311 [=

311/311 [==============================] - 3s 9ms/step - loss: 1.8755e-04
Epoch 2374/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8839e-04
Epoch 2375/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8809e-04
Epoch 2376/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8793e-04
Epoch 2377/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8778e-04
Epoch 2378/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8756e-04
Epoch 2379/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8840e-04
Epoch 2380/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8810e-04
Epoch 2381/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8793e-04
Epoch 2382/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8779e-04
Epoch 2383/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8757e-04
Epoch 2384/2500
311/311 [=

311/311 [==============================] - 3s 9ms/step - loss: 1.8793e-04
Epoch 2465/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8772e-04
Epoch 2466/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8847e-04
Epoch 2467/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8816e-04
Epoch 2468/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8799e-04
Epoch 2469/2500
311/311 [==============================] - 3s 8ms/step - loss: 1.8774e-04A: 0s - loss: 1.867
Epoch 2470/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8858e-04
Epoch 2471/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8827e-04
Epoch 2472/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8809e-04
Epoch 2473/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8792e-04
Epoch 2474/2500
311/311 [==============================] - 3s 9ms/step - loss: 1.8774e-04
Epoch 2

In [21]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 0.31 RMSE
